In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc, cv2
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
traindf = pd.read_csv('../input/datacsv/train.csv', dtype=str)
testdf = pd.read_csv('../input/datacsv/test.csv', dtype=str)

In [ ]:
traindf['image_id'] = traindf['image_id'] + ".jpg"
testdf['image_id'] = testdf['image_id'] + ".jpg"

In [ ]:
datagen = ImageDataGenerator(rescale=1./255.,
                             rotation_range=40,
                             width_shift_range=0.2, 
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True,
                             fill_mode='nearest')

train_generator = datagen.flow_from_dataframe(
dataframe=traindf,
directory="",
x_col="image_id",
y_col="category",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224, 224))

In [ ]:
tabLabels=[]
for i in range(1, 103):
    tabLabels.append(i)
labels = tabLabels
# labels = dict((v,k) for k,v in labels.items())
len(labels)

In [ ]:
vgg_model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (224, 224, 3))

for layer in vgg_model.layers[:-5]:
    layer.trainable=False
    
for layer in vgg_model.layers[1:4]:
    layer.trainable=True
    
input = Input(shape=(224, 224, 3),name = 'image_input')
output_vgg16_conv = vgg_model(input)

x = BatchNormalization()(output_vgg16_conv)
x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
x = Dropout(0.2)(x)

x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(102, activation='softmax')(x)

model = Model(inputs=input, outputs=x)

model.summary()

In [ ]:
model.load_weights("../input/flowers-vgg19-weights/flowers_vgg19.h5")

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np

def output(location, new_model):
    img = load_img(location, target_size = (224, 224, 3))
    img = img_to_array(img)
    img = img / 255
    img = np.expand_dims(img, axis=0)
    answer = new_model.predict(img)
    y_class = answer.argmax(axis = -1)
    label = labels[np.argmax(answer)]
    print('label is', label)
    
    top_3 = np.argsort(answer[0])[:-4:-1]
#     for i in range(3):
#         print(" ({:.3})".format(answer[0][top_3[i]]))
    
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res = labels[y]
    print("category", res)
#     print(" ({:.3})".format(answer[0][top_3[0]]))
    pred_prob = answer[0][top_3[0]]
    print(" ({:.3})".format(pred_prob))
    
#     print ("new ********************")   
#     proba = answer[0]
#     idxs = np.argsort(proba)[::-1][:3]
#     for (i, j) in enumerate(idxs):
# #         y = labels[i]        
#         print(labels[i])
#     print("new prob", proba)
    return float(pred_prob), label

In [ ]:
location = "../input/classes/flowers_classes_top/1/100.jpg"
output(location, model)

In [ ]:
import cv2
import os
from shutil import copyfile
from pathlib import Path

#folder="/content/drive/My Drive/Colab/1KDataSet_Part1"
folder = "../input/classes/flowers_classes_top/"

last_fileName=""
#images = []
imagesFileNames = []
for filename in os.listdir(folder):
    # for img in os.listdir(folder+filename):
    imagesFileNames.append(filename)
    # print(filename)
print(len(imagesFileNames))

for i in range(0, len(imagesFileNames)) :
    imagesFileNames[i] = int(imagesFileNames[i])
imagesFileNames.sort()

# imagesFileNames = selection_sort(imagesFileNames)
# imagesFileNames.sort()
for i in range(0, len(imagesFileNames)) :
    imagesFileNames[i] = str(imagesFileNames[i])
imagesFileNames

In [ ]:
arryWeights_last_After =[]
for i in range(512):
  arryWeights_last_After.append(i)
for i in range(512):
  arryWeights_last_After[i]=1

def custom_layer_last(tensor):
    return tensor * arryWeights_last_After[i]
# vgg_weights = VGG16(    input_shape = IMAGE_SIZE + [3], weights = 'imagenet',
#     include_top = False).get_weights()

In [ ]:
def insert_intermediate_layer_in_keras(model, layer_id, new_layer):
    from tensorflow.keras.models import Model

    layers = [l for l in model.layers]

    x = layers[0].output
    for i in range(1, len(layers)):
        if i == layer_id:
            x = new_layer(x)
        x = layers[i](x)

    model = Model(inputs=layers[0].input, outputs=x) ### inputs instead of input in this version
    return model

In [ ]:
def build_model():
    vgg_model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (224, 224, 3))

    for layer in vgg_model.layers[:-5]:
        layer.trainable=False

    for layer in vgg_model.layers[1:4]:
        layer.trainable=True

    input = Input(shape=(224, 224, 3),name = 'image_input')
    output_vgg16_conv = vgg_model(input)

    x = BatchNormalization()(output_vgg16_conv)
    x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(102, activation='softmax')(x)

    model = Model(inputs=input, outputs=x)

    return model

# This part for computing the CI and storing it into Dictionnary in tx file

In [ ]:
from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, MaxPool2D 

# from kerassurgeon import Surgeon
# from kerassurgeon import identify
# from kerassurgeon import utils
# from kerassurgeon.operations import delete_channels
# dctOfAllDictionClasses = {}

weights_dic = {}

imgCount = 0
indexImg=0
# img_path = 'Mountain_Bike/[image-net.org][27]443691662_09f12b8e37.jpg'
for imgFileName in imagesFileNames:
    dctOfAllDictionClasses = {}
    imgPath = folder + imagesFileNames[indexImg] + "/" + os.listdir(folder + imagesFileNames[indexImg])[0]
    print(imgPath)
    res = output(imgPath, model)
    print('res[0]', res[0])
    print('res[1]', res[1])
    originalName = imgFileName
    # originalName = res[1]
    # if '.jpg' in originalName:
    #     originalName = originalName.replace('.jpg','')
    print('orign name:', originalName)
    class_Id = res[1]
    orig_acc = res[0]
    print('origin acc', orig_acc)
    weights_dic = {}
    K.set_learning_phase(0)
    
    import time
    start_time = time.time()
       
    for i in range(512):
        print ('filt{}'.format (i))
        arryWeights_last_After[i]=0
        # def custom_layer_last(tensor):
        #   return tensor * arryWeights_last_After[i]
        filt = i
#         #print(model.summary())
#         #model = insert_intermediate_layer_in_keras(model, 4, BatchNormalization()
        def custom_layer_last(tensor):
            # print ('tesnor values :', tensor[0][0])
            return tensor * arryWeights_last_After # working version

#         new_model = build_model()
        lambda_layer = Lambda (custom_layer_last, name="lambda_New")
        new_model = build_model()
        new_model.load_weights('../input/flowers-vgg19-weights/flowers_vgg19.h5')

        new_model.layers[1] = insert_intermediate_layer_in_keras (new_model.layers[1], 21, lambda_layer)

        img = load_img(imgPath, target_size = (224, 224, 3))
        img = img_to_array(img)
        img = img / 255
        img = np.expand_dims(img, axis=0)
        answer = new_model.predict(img)
        y_class = answer.argmax(axis = -1)

        top_3 = np.argsort(answer[0])[:-4:-1]
        New_accu = answer[0][top_3[0]]
        print(" ({:.3})".format(New_accu))
        print('label name:', res)
        print ('acc after purning (%.2f%%)', New_accu)
        print('difference: ', (orig_acc - New_accu))
        #weights_dic[filt][0] = 1/(k+1)  # resp
        tuple = orig_acc - New_accu, 1
        #weights_dic[filt] = original_loss[1] - loss[1]  # prob
        weights_dic[filt] = tuple
        arryWeights_last_After[i]=1

    weights_dic_sort = sorted (weights_dic.items (), key=lambda kv: kv[1], reverse=True)
    #     print ('Resp and loss for conv layer {}\n'.format (1), weights_dic_sort)
    # arryOfDictFiltByLayer.append(weights_dic_sort)
    #K.backend.clear_session ()
    print("--- %s seconds ---" % (time.time() - start_time))
    pairKeyValue = { originalName: weights_dic_sort}
    dctOfAllDictionClasses = {}
    dctOfAllDictionClasses.update(pairKeyValue)
    print('pair ley:', pairKeyValue)
    import json
    #     with open('FIltersRespAll', 'w') as fout:
    #         json.dump(str(lstOfAllDictionClasses), fout)
    if os.path.exists('../output/kaggle/working/Filters_Resp_Flowers.txt'):
        append_write = 'a' # append if already exists
    else:
        append_write = 'w+' # make a new file if not
    with open('../output/kaggle/working/Filters_Resp_Flowers.txt', append_write) as fout:
        fout.write("\n" + str(dctOfAllDictionClasses) + "\n") 
        print('Last Image ID:', imagesFileNames[imgCount])
    imgCount+=1
    indexImg+=1

